# 3DVar(3次元変分法)
予報誤差共分散を更新しない．速い．

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# モジュールの読み込み
import sys
sys.path.append('./module')
from utils import make_lorenz96, rk4, error_series_kf, plot_error_KF, estimate_error_with_params

from kalman_filters import EnsembleKalmanFilter as ExKF

In [5]:
# Lorenz96の設定
J = 40
F = 8
lorenz = make_lorenz96(F)

# 同化step
# 時間発展は0.01ごとに行う
dt = 0.05

# モデルの遷移関数(非線形)
# 0.01ずつ時間発展させる
# dtは同化step
def M(x, dt):
    for i in range(int(dt/0.01)):
        x = rk4(0, x, 0.01, lorenz)
    return x

# 単位行列
I = np.identity(J)

# 観測
H = I

# モデル誤差共分散, 最初は完全モデルを仮定
Q = np.zeros((J, J))

# 観測誤差共分散, 後で定数倍の変化をさせる.
R = I

# 観測値と真値
end_step = 500 # 開発用
y = np.load('data/obs_atr.npy')
true = np.load('data/true_atr.npy')

# KFの初期値
np.random.seed(1)
x_0 = true[np.random.randint(len(true)-1)]
P_0 = 25*I

NameError: name 'make_lorenz96' is not defined

In [4]:
%%time
# 実行済
# 予報誤差共分散の初期値更新しないので小さくしておく
P_00 = I

var3d = ExKF(M, H, Q, R, y, x_0, P_00, dt=dt, var3d=True)
var3d.forward_estimation()
np.save('data/var3d/var3d.npy', var3d.x)

NameError: name 'I' is not defined

In [3]:
var3d_x = np.load('data/var3d/var3d.npy')
plot_error_KF(true, y, [var3d_x])

FileNotFoundError: [Errno 2] No such file or directory: 'data/var3d/var3d.npy'

観察
- 観測データよりは良い推定を安定して行うことができている．

## 3DVarの観測分布への依存性を調べる

cut size: 0 ~ 40
各cut sizeで1サンプルの平均をとる

In [ ]:
# 実行済
# %%time
# results = []
# max_cut_size = 40
# n_samples = 1
# for cut_obs_size in range(max_cut_size+1):
#     print('===========\n')
#     print('cut_obs_size: {}'.format(cut_obs_size))
    
#     var3d_random_cut_obs_samples = []
#     for n in range(n_samples):
#         print('---------------')
#         print('n: {}'.format(n))
        
#         var3d_random_cut_obs = ExKF(M, H, Q, R, y, x_0, P_0=I, dim_x=J, dt=dt, var3d=True, cut_obs_size=cut_obs_size)
#         var3d_random_cut_obs.forward_estimation()
#         var3d_random_cut_obs_samples.append(var3d_random_cut_obs.x)
#     results.append(np.array(var3d_random_cut_obs_samples).mean(axis=0))
# np.save('data/var3d/var3d_random_cut_obs_{}parametrized_{}samples.npy'.format(max_cut_size, n_samples), results)

In [ ]:
max_cut_size = 40
n_samples = 1
var3d_random_cut_obs_results = np.load('data/var3d/var3d_random_cut_obs_{}parametrized_{}samples.npy'.format(max_cut_size, n_samples))
params_cut_obs = range(max_cut_size+1)
legends = ['cut size = {}'.format(n) for n in params_cut_obs]
plot_error_KF(true, y, var3d_random_cut_obs_results, legends)
estimate_error_with_params(true, var3d_random_cut_obs_results, params_cut_obs, 'cut size ({}sample mean)'.format(n_samples), log=False)

### 観察
- 1min程度かかった
- 観測を減らす数が30あたりから急激にrmseが上昇している．
- 観測が0になるとrmseはアトラクター平均距離(５くらい)になっている．

## 3次元変分法のパラメータ依存性

1. $R = rI$として$r$を変化させる．
2. $P_0 = p_0 I$として$p_0$を変化させる．

#### 仮説
- $r$を大きくするとモデルに近づく. 
- $p_0$を大きくするとデータに近づく．


### 観測誤差$R$を変化させる
- $r$の値を指数的に増加させる．

In [ ]:
# 実行済
# results_r = []
# params_r = []
# P_00 = I
# for k in range(7):
#     r = 10**(k-2)
#     params_r.append(r)
#     R_r = r*I
#     var3d_r = ExKF(M, H, Q, R_r, y, x_0, P_00, dim_x=J, dt=dt, var3d=True)
#     var3d_r.forward_estimation()
#     results_r.append(var3d_r.x)
# np.save('data/var3d/var3d_r_parametrized.npy', np.array(results_r))

In [ ]:
vard3_results_r = np.load('data/var3d/var3d_r_parametrized.npy')
params_r = [10**(k-2) for k in range(7)]
legends_r = ['r = {}'.format(r) for r in params_r]
plot_error_KF(true, y, vard3_results_r, legends = legends_r)
estimate_error_with_params(true, vard3_results_r, params_r, 'r', log=True)

### 観察
- $r$が小さいと推定が観測データに近づく． 観測誤差~ 推定誤差．
- $r$を大きくすると推定誤差が小さくなるがある値を超えると観測誤差を超えて大きくなる．

### 誤差共分散$P_0$を変化させる
- $p_0$の値を指数的に増加させる．

In [ ]:
# 実行済
# results_var3d_p = []
# params_p = []
# for k in range(7):
#     p = 10**(k-4)
#     params_p.append(p)
#     P_00 = p*I
#     var3d_r = ExKF(M, H, Q, R, y, x_0, P_0=P_00, dim_x=J, dt=dt, var3d=True)
#     var3d_r.forward_estimation()
#     results_var3d_p.append(var3d_r.x)
# np.save('data/var3d/var3d_p_parametrized.npy', np.array(results_var3d_p))

In [ ]:
vard3_results_p = np.load('data/var3d/var3d_p_parametrized.npy')
params_p = [10**(k-4) for k in range(7)]
legends_p = ['p_0 = {}'.format(p) for p in params_p]
plot_error_KF(true, y, vard3_results_p, legends = legends_p)
estimate_error_with_params(true, vard3_results_p, params_p, 'p', log=True)

### 観察
- $p_0$が小さいと観測誤差を超えて大きい推定誤差になっている．
- $p_0$を大きくすると推定誤差が小さくなるがある値を超えると観測誤差1に近づく．
- $r$を変化させた時と逆の挙動をしている．

## $P_0 = p_0 I$の最適化

In [ ]:
# %%time
# results_var3d_p = []
# params_p = []
# for k in range(11):
#     p = 0.23 - 0.01 + 0.002*k
#     params_p.append(p)
#     P_00 = p*I
#     var3d_r = ExKF(M, H, Q, R, y, x_0, P_0=P_00, dt=dt, var3d=True)
#     var3d_r.forward_estimation()
#     results_var3d_p.append(var3d_r.x)
# np.save('data/var3d/var3d_p_detail_parametrized.npy', np.array(results_var3d_p))

In [ ]:
vard3_results_p = np.load('data/var3d/var3d_p_detail_parametrized.npy')
params_p = [0.23 - 0.01 + 0.002*k for k in range(11)]
legends_p = ['p_0 = {}'.format(p) for p in params_p]
plot_error_KF(true, y, vard3_results_p, legends = legends_p)
estimate_error_with_params(true, vard3_results_p, params_p, 'p')

### 観察
- $0.002$ 刻みでは$ p_0 = 0.228 $が最適
- rmseは0.3871
- 実行速度は: 1.49s